### 4-3-2 オブジェクトへの変更を禁止する

In [2]:
o = Object.new
o.freeze
o.frozen?

true

In [3]:
o2 = Object.new
o2.frozen?

false

In [4]:
str = "hoge"
str.frozen?

false

In [5]:
str << "2"

"hoge2"

In [6]:
str.freeze
str << "0"

RuntimeError: can't modify frozen String

`.frreeze`しても代入は出来てしまう

In [7]:
DEFAULT_CONST = {foo: 123}.freeze
DEFAULT_CONST = {foo: 234}

(pry):14: warning: already initialized constant DEFAULT_CONST
(pry):13: warning: previous definition of DEFAULT_CONST was here


{:foo=>234}

In [8]:
DEFAULT_CONST.frozen?

false

In [9]:
DEFAULT_CONST.freeze
DEFAULT_CONST.frozen?

true

`Hash`の中身は変更できない

In [10]:
DEFAULT_CONST[:foo] = 2

RuntimeError: can't modify frozen Hash

In [11]:
FROZEN_INT = 3
FROZEN_INT.freeze
FROZEN_INT += 3

(pry):21: warning: already initialized constant FROZEN_INT
(pry):19: warning: previous definition of FROZEN_INT was here


6

In [12]:
:foo = 3
:foo
:foo.frozen?

SyntaxError: unexpected '=', expecting end-of-input
:foo = 3
      ^


In [13]:
:foo.extend Enumerable

TypeError: can't define singleton

### 4-3-3 オブジェクトをコピーする

In [14]:
original = Object.new
original.object_id

70282404920840

In [15]:
original.freeze
copy_dup = original.dup
p copy_dup.object_id
copy_dup.frozen?

70282396473120


false

In [16]:
copy_clone = original.clone
p copy_clone.object_id
copy_clone.frozen?

70282404790600


true

In [17]:
value = 'foo'
array = [value]
array_dup = array.dup
array_clone = array.clone
p value.object_id
p array_dup.first.object_id
p array_clone.first.object_id

70282408680320
70282408680320
70282408680320


70282408680320

In [18]:
p array
p array_dup

["foo"]
["foo"]


["foo"]

In [19]:
array_dup[0] = "bar"
p array_dup
p array

["bar"]
["foo"]


["foo"]

In [20]:
array_dup = array.dup
array_dup
array_dup[0] << '2'
p array_dup
array

["foo2"]


["foo2"]

安全にdeep copyするなら`Marshal.dump`, `Marshal.load`するのも一つの方法

In [21]:
Marshal.dump(array)
array_dump= Marshal.load(Marshal.dump(array))
array_dump.first.object_id

70282409381020

In [22]:
array.first.object_id

70282408680320

In [23]:
array_dump[0] << '0'
p array_dump
array

["foo20"]


["foo2"]

### 4-3-4 汚染されたオブジェクト

pryとirubyの挙動が違う...

In [24]:
$SAFE

0

In [25]:
Object.new.tainted?

false

In [26]:
ENV['HOME'].tainted?

true

In [27]:
o = Object.new
p o.tainted?
o.taint
p o.tainted?

false
true


true

In [28]:
rcfile = ENV['HOME'] + '/.zshrc'
rcfile.tainted?

true

In [29]:
$SAFE = 1

1

In [30]:
$SAFE

1

In [31]:
test_file = ENV['HOME'] + '/test_file'

"/Users/mic/test_file"

In [32]:
ENV['HOME'].class

String

In [33]:
ENV['HOME'].tainted?

true

# 5章 主な組込みクラス／モジュール
## 5-1 Numeric

In [34]:
p Integer.ancestors
p Float.ancestors
p Rational.ancestors
p Complex.ancestors
p Fixnum.ancestors
Bignum.ancestors

[Integer, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]
[Float, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]
[Rational, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]
[Complex, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]
[Fixnum, Integer, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]


[Bignum, Integer, Numeric, Comparable, Object, PP::ObjectMixin, Kernel, BasicObject]

In [38]:
p 0.zero?
p 3.zero?

true
false


false

In [39]:
ary = [1, 0, 3]
p ary.select(&:zero?)
p ary.reject(&:zero?)
ary.select(&:nonzero?)

[0]
[1, 3]


[1, 3]

In [41]:
p 365.integer?
p 36.5.integer?
p 36.5.numeric?

true
false


NoMethodError: undefined method `numeric?' for 36.5:Float

In [43]:
p 36.5.real?
p 100.bignum?

true


NoMethodError: undefined method `bignum?' for 100:Fixnum

In [44]:
p Complex(1, 2).real?

false


false

### 5-1-1 算術演算

In [46]:
p 10 /3
p 10.0 /3
num = 10
p num / 3
num.to_f / 3

3
3.3333333333333335
3


3.3333333333333335

### 5-1-2 比較演算

In [47]:
123 == 123.0

true

### 5-1-3 丸め操作

In [48]:
p 1.4.round
p 1.5.round
p -1.5.round

1
2
-2


-2

In [49]:
p 1.4.ceil
p 1.5.ceil
p -1.5.ceil
p -1.4.ceil

2
2
-1
-1


-1

In [50]:
p 1.5.floor
p 1.4.floor
p -1.4.floor

1
1
-2


-2

In [52]:
p 1.5.truncate
p -1.5.truncate
p 1.6.truncate
p 1.7.truncate
p 1.9.truncate
p -0.9.truncate
p -1.9.truncate

1
-1
1
1
1
0
-1


-1

### 5-1-4 繰り返し処理

In [53]:
3.step 5 do |num|
  puts num
end

3
4
5


3

In [54]:
1.2.step 2.0, 0.2 do |num|
  puts num
end

1.2
1.4
1.6
1.8
2.0


1.2

In [55]:
3.upto(5) do |num|
  puts num
end

3
4
5


3

In [56]:
3.upto(10, 2) do |num|
  puts num
end

ArgumentError: wrong number of arguments (2 for 1)

In [57]:
1.3.upto(2.0) do |num|
  puts
end

NoMethodError: undefined method `upto' for 1.3:Float

次回は、`5-1-5 Integer`から